# Snapshot LUNA

In [9]:
import json
from collections import defaultdict
from pathlib import Path
from time import time

import numpy as np
from utils.rpc import w3
from utils.constants import *

In [10]:
def zero():
    return 0


def save(file, dic):
    with open(file, "w") as f:
        json.dump(dic, f)

def getTokenBalance(tokenName):
    tokenAddress = w3.toChecksumAddress(tokens[tokenName])
    token = w3.eth.contract(address=tokenAddress, abi=ERC20_ABI)

    bal_pre = {}
    bal_post = {}

    for pairAddress in pairs.values():
        bal_pre[pairAddress] = token.functions.balanceOf(w3.toChecksumAddress(pairAddress)).call({}, PRE_ATTACK_BLOCK)
        bal_post[pairAddress] = token.functions.balanceOf(w3.toChecksumAddress(pairAddress)).call({}, POST_ATTACK_BLOCK)

    return bal_pre, bal_post

In [3]:
tokens_balances_pre_attack = {}
tokens_balances_post_attack = {}
t0 = time()
for name in tokens.keys():
    tokens_balances_pre_attack[name], tokens_balances_post_attack[name] = getTokenBalance(name)
print("Done in {}s".format(round(time() - t0, 2)))

Done in 23.07s


In [11]:
def getPairBalance(name):
    with open("json/pairs/{}.json".format(name), "r") as f:
        txs = json.load(f)
    bal_pre = defaultdict(zero)
    bal_post = defaultdict(zero)
    
    for tx in txs:
        # ignore the LP sent to MCJV3
        if tx["from"] == MCJV3_ADDRESS or tx["to"] == MCJV3_ADDRESS:
            continue
    
        if tx["block"] <= PRE_ATTACK_BLOCK:
            bal_pre[tx["from"]] -= tx["amount"]
            bal_pre[tx["to"]] += tx["amount"]
        if tx["block"] <= POST_ATTACK_BLOCK:
            bal_post[tx["from"]] -= tx["amount"]
            bal_post[tx["to"]] += tx["amount"]

    assert np.all(
        np.array([amount for user, amount in bal_pre.items() if user != ADDRESS_ZERO]) >= 0
    )
    assert np.all(
        np.array([amount for user, amount in bal_post.items() if user != ADDRESS_ZERO]) >= 0
    )
    assert sum(bal_pre.values()) == 0
    assert sum(bal_post.values()) == 0
    
    return bal_pre, bal_post

In [12]:
pairs_balances_pre_attack = {}
pairs_balances_post_attack = {}
for name in pairs.keys():
    pairs_balances_pre_attack[name], pairs_balances_post_attack[name] = getPairBalance(name)

In [13]:
pre_attack_balance = {}
for symbol, token_balances in tokens_balances_pre_attack.items():
    pre_attack_balance[symbol] = defaultdict(zero)
    sum_balance = 0
    for name, lp_balances in pairs_balances_pre_attack.items():
        if ADDRESS_ZERO in lp_balances:
            pair_balance = token_balances[pairs[name]]
            total_lp_supply = -lp_balances[ADDRESS_ZERO]
            if pair_balance != 0:
                sum_balance += pair_balance
                for user, lp_balance in lp_balances.items():
                    if user != ADDRESS_ZERO and lp_balance != 0:
                        pre_attack_balance[symbol][user] += (
                            lp_balance * pair_balance // total_lp_supply
                        )
    assert sum(pre_attack_balance[symbol].values()) <= sum_balance

pre_attack_balance = {name: sorted(balances.items(), key=lambda x: x[1], reverse=True)
                      for name, balances in pre_attack_balance.items()}

post_attack_balance = {}
for symbol, token_balances in tokens_balances_post_attack.items():
    post_attack_balance[symbol] = defaultdict(zero)
    sum_balance = 0
    for name, lp_balances in pairs_balances_post_attack.items():
        if ADDRESS_ZERO in lp_balances:
            pair_balance = token_balances[pairs[name]]
            total_lp_supply = -lp_balances[ADDRESS_ZERO]
            if pair_balance != 0:
                sum_balance += pair_balance
                for user, lp_balance in lp_balances.items():
                    bal = lp_balance * pair_balance // total_lp_supply
                    if user != ADDRESS_ZERO and bal != 0:
                        post_attack_balance[symbol][user] += (
                            lp_balance * pair_balance // total_lp_supply
                        )
    assert sum(post_attack_balance[symbol].values()) <= sum_balance

post_attack_balance = {name: {k:v for k,v in sorted(balances.items(), key=lambda x: x[1], reverse=True)}
                      for name, balances in post_attack_balance.items()}

In [14]:
# Create paths if they don't exist yet
Path("json/snapshot/pre-attack").mkdir(parents=True, exist_ok=True)
Path("json/snapshot/post-attack").mkdir(parents=True, exist_ok=True)

for symbol, dic in pre_attack_balance.items():
    save("json/snapshot/pre-attack/{}.json".format(symbol), dic)

for symbol, dic in post_attack_balance.items():
    save("json/snapshot/post-attack/{}.json".format(symbol), dic)